# Twitter Bot Detector
In this notebook, we are using a dataset from [Kaggle](https://www.kaggle.com/davidmartngutirrez/twitter-bots-accounts) and [Botometer](https://botometer.osome.iu.edu/bot-repository/datasets.html), containing in total 80K twitter user ids. Thanks to the Tweepy Python library we are retrieving users' information, which we considered useful in order to determine the type of an account.We have numerical (followers,friends,listed,statuses,retweets,urls,mentions,age of the account), categorical (geo enabled,verified,has extended profile, has default profile,has default profile image) and also textual (description,tweets) features. So we decided to train 3 models, one for the numerical/categorical features, another for the textual features and one more for the combination of the two types of features.
We also used different classifiers and different evaluation metrics, in order to make comparisons between the results.

## Install Dependencies

In [0]:
!sudo pip install --upgrade pip
!sudo pip install pyspark --upgrade
!pip install nltk
!pip install stopwords
!pip install mlflow
!pip install tweepy

Collecting pip
 Downloading pip-21.1.1-py3-none-any.whl (1.5 MB)
▏ | 10 kB 14.2 MB/s eta 0:00:01 |▍ | 20 kB 13.1 MB/s eta 0:00:01 |▋ | 30 kB 6.5 MB/s eta 0:00:01 |▉ | 40 kB 6.2 MB/s eta 0:00:01 |█ | 51 kB 6.4 MB/s eta 0:00:01 |█▎ | 61 kB 6.2 MB/s eta 0:00:01 |█▌ | 71 kB 6.8 MB/s eta 0:00:01 |█▊ | 81 kB 7.1 MB/s eta 0:00:01 |██ | 92 kB 6.3 MB/s eta 0:00:01 |██▏ | 102 kB 6.7 MB/s eta 0:00:01 |██▎ | 112 kB 6.7 MB/s eta 0:00:01 |██▌ | 122 kB 6.7 MB/s eta 0:00:01 |██▊ | 133 kB 6.7 MB/s eta 0:00:01 |███ | 143 kB 6.7 MB/s eta 0:00:01 |███▏ | 153 kB 6.7 MB/s eta 0:00:01 |███▍ | 163 kB 6.7 MB/s eta 0:00:01 |███▋ | 174 kB 6.7 MB/s eta 0:00:01 |███▉ | 184 kB 6.7 MB/s eta 0:00:01 |████ | 194 kB 6.7 MB/s eta 0:00:01 |████▎ | 204 kB 6.7 MB/s eta 0:00:01 |████▌ | 215 kB 6.7 MB/s eta 0:00:01 |████▋ | 225 kB 6.7 MB/s eta 0:00:01 |████▉ | 235 kB 6.7 MB/s eta 0:00:01 |█████ | 245 kB 6.7 MB/s eta 0:00:01 |█████▎ | 256 kB 6.7 MB/s eta 0:00:01 |█████▌ | 266 kB 6.7 MB/s eta 0:00:01 |█████▊ | 276 kB 6.7 MB/s eta 0:00:01 |██████ | 286 kB 6.7 MB/s eta 0:00:01 |██████▏ | 296 kB 6.7 MB/s eta 0:00:01 |██████▍ | 307 kB 6.7 MB/s eta 0:00:01 |██████▋ | 317 kB 6.7 MB/s eta 0:00:01 |██████▊ | 327 kB 6.7 MB/s eta 0:00:01 |███████ | 337 kB 6.7 MB/s eta 0:00:01 |███████▏ | 348 kB 6.7 MB/s eta 0:00:01 |███████▍ | 358 kB 6.7 MB/s eta 0:00:01 |███████▋ | 368 kB 6.7 MB/s eta 0:00:01 |███████▉ | 378 kB 6.7 MB/s eta 0:00:01 |████████ | 389 kB 6.7 MB/s eta 0:00:01 |████████▎ | 399 kB 6.7 MB/s eta 0:00:01 |████████▌ | 409 kB 6.7 MB/s eta 0:00:01 |████████▊ | 419 kB 6.7 MB/s eta 0:00:01 |█████████ | 430 kB 6.7 MB/s eta 0:00:01 |█████████ | 440 kB 6.7 MB/s eta 0:00:01 |█████████▎ | 450 kB 6.7 MB/s eta 0:00:01 |█████████▌ | 460 kB 6.7 MB/s eta 0:00:01 |█████████▊ | 471 kB 6.7 MB/s eta 0:00:01 |██████████ | 481 kB 6.7 MB/s eta 0:00:01 |██████████▏ | 491 kB 6.7 MB/s eta 0:00:01 |██████████▍ | 501 kB 6.7 MB/s eta 0:00:01 |██████████▋ | 512 kB 6.7 MB/s eta 0:00:01 |██████████▉ | 522 kB 6.7 MB/s eta 0:00:01 |███████████ | 532 kB 6.7 MB/s eta 0:00:01 |███████████▏ | 542 kB 6.7 MB/s eta 0:00:01 |███████████▍ | 552 kB 6.7 MB/s eta 0:00:01 |███████████▋ | 563 kB 6.7 MB/s eta 0:00:01 |███████████▉ | 573 kB 6.7 MB/s eta 0:00:01 |████████████ | 583 kB 6.7 MB/s eta 0:00:01 |████████████▎ | 593 kB 6.7 MB/s eta 0:00:01 |████████████▌ | 604 kB 6.7 MB/s eta 0:00:01 |████████████▊ | 614 kB 6.7 MB/s eta 0:00:01 |█████████████ | 624 kB 6.7 MB/s eta 0:00:01 |█████████████▏ | 634 kB 6.7 MB/s eta 0:00:01 |█████████████▍ | 645 kB 6.7 MB/s eta 0:00:01 |█████████████▌ | 655 kB 6.7 MB/s eta 0:00:01 |█████████████▊ | 665 kB 6.7 MB/s eta 0:00:01 |██████████████ | 675 kB 6.7 MB/s eta 0:00:01 |██████████████▏ | 686 kB 6.7 MB/s eta 0:00:01 |██████████████▍ | 696 kB 6.7 MB/s eta 0:00:01 |██████████████▋ | 706 kB 6.7 MB/s eta 0:00:01 |██████████████▉ | 716 kB 6.7 MB/s eta 0:00:01 |███████████████ | 727 kB 6.7 MB/s eta 0:00:01 |███████████████▎ | 737 kB 6.7 MB/s eta 0:00:01 |███████████████▌ | 747 kB 6.7 MB/s eta 0:00:01 |███████████████▊ | 757 kB 6.7 MB/s eta 0:00:01 |███████████████▉ | 768 kB 6.7 MB/s eta 0:00:01 |████████████████ | 778 kB 6.7 MB/s eta 0:00:01 |████████████████▎ | 788 kB 6.7 MB/s eta 0:00:01 |████████████████▌ | 798 kB 6.7 MB/s eta 0:00:01 |████████████████▊ | 808 kB 6.7 MB/s eta 0:00:01 |█████████████████ | 819 kB 6.7 MB/s eta 0:00:01 |█████████████████▏ | 829 kB 6.7 MB/s eta 0:00:01 |█████████████████▍ | 839 kB 6.7 MB/s eta 0:00:01 |█████████████████▋ | 849 kB 6.7 MB/s eta 0:00:01 |█████████████████▉ | 860 kB 6.7 MB/s eta 0:00:01 |██████████████████ | 870 kB 6.7 MB/s eta 0:00:01 |██████████████████▏ | 880 kB 6.7 MB/s eta 0:00:01 |██████████████████▍ | 890 kB 6.7 MB/s eta 0:00:01 |██████████████████▋ | 901 kB 6.7 MB/s eta 0:00:01 |██████████████████▉ | 911 kB 6.7 MB/s eta 0:00:01 |███████████████████ | 921 kB 6.7 MB/s eta 0:00:01 |███████████████████▎ | 931 kB 6.7 MB/s eta 0:00:01 |███████████████

## Import Python Packages

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.functions import udf
from pyspark.sql.session import SparkSession
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.classification import FMClassifier , GBTClassifier
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler, OneHotEncoder
from pyspark.ml.feature import StopWordsRemover, Word2Vec, RegexTokenizer, Tokenizer, MinMaxScaler
from pyspark.sql.functions import udf, col, lower, trim, regexp_replace

import datetime
import nltk
from nltk.stem.snowball import SnowballStemmer 
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 
import re
from IPython.display import HTML
now = datetime.datetime.now()
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))  
porter = PorterStemmer()
RANDOM_SEED = 42 # for reproducibility

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data] Unzipping corpora/stopwords.zip.

In [0]:
import tweepy
from tweepy import OAuthHandler

consumer_key = "phCKVDVUS7nBmCvN5aJZWwrxo"
consumer_secret = "3k7gMiVmxPPDI0C6kTc8uMTL0nSdNfeeU82OGcNVftkaMmujlR"
access_token = "1389540894022545408-lovri9oSZKdLryO5JXqvfwuLeruEGq"
access_token_secret = "N0XU2cNl14QsO13IRDHCSFmuZFpELKqNG2pA9mkwaQfrg"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [0]:
s = """<!DOCTYPE html>
<html>
	<head>
		<title>Bot Detector Website</title>
        <link rel= "stylesheet" type= "text/css" href= "{{ url_for('static',filename='styles/index.css') }}">
	</head>
	<body>
        <div class="container">
            <h1>Twitter Bot Detector</h1> 
            <p>Check if a user is human or bot.</p>
            <form action="" method="post">
                <input type="text" placeholder="e.g. @hello_kitty123" name="name">
                <div class="bar"></div>
                <div class="highlight"></div>
                <button class="btn striped-shadow dark" type="submit" value="submit"><span>Check</span></button>
            </form>

        </div>
	</body>
    
    <style>
body {
  font-family: courier, arial, helvetica;
  font-size: 150%;
  color: #77bfa1;
  background-color: #726da8;
}

.container {
  text-align: center;
  margin-top: 10%;
  font-weight: bold;
}

@import "https://fonts.googleapis.com/css?family=Bungee+Shade";

*,
:after,
:before {
  box-sizing: border-box;
}

:focus {
  outline: none;
}

button {
  overflow: visible;
  border: 0;
  padding: 0;
  margin: 1.8rem;
}
.btn.striped-shadow span {
  display: block;
  position: relative;
  z-index: 2;
  border: 5px solid;
}

.btn.striped-shadow.dark span {
  border-color: #393939;
  background: #77bfa1;
  color: #393939;
}

.btn {
  height: 80px;
  line-height: 65px;
  display: inline-block;
  letter-spacing: 1px;
  position: relative;
  font-size: 1.35rem;
  transition: opacity 0.3s, z-index 0.3s step-end, -webkit-transform 0.3s;
  transition: opacity 0.3s, z-index 0.3s step-end, transform 0.3s;
  transition: opacity 0.3s, z-index 0.3s step-end, transform 0.3s,
    -webkit-transform 0.3s;
  z-index: 1;
  background-color: transparent;
  cursor: pointer;
}

.btn {
  width: 155px;
  height: 48px;
  line-height: 38px;
}

button.btn.striped-shadow.dark:after,
button.btn.striped-shadow.dark:before {
  background-image: linear-gradient(
    135deg,
    transparent 0,
    transparent 5px,
    #393939 5px,
    #393939 10px,
    transparent 10px
  );
}

button.btn.striped-shadow:hover:before {
  max-height: calc(100% - 10px);
}

button.btn.striped-shadow:after {
  width: calc(100% - 4px);
  height: 8px;
  left: -10px;
  bottom: -9px;
  background-size: 15px 8px;
  background-repeat: repeat-x;
}
button.btn.striped-shadow:after,
button.btn.striped-shadow:before {
  content: "";
  display: block;
  position: absolute;
  z-index: 1;
  transition: max-height 0.3s, width 0.3s, -webkit-transform 0.3s;
  transition: transform 0.3s, max-height 0.3s, width 0.3s;
  transition: transform 0.3s, max-height 0.3s, width 0.3s,
    -webkit-transform 0.3s;
}

.btn.striped-shadow:hover {
  -webkit-transform: translate(-12px, 12px);
  -ms-transform: translate(-12px, 12px);
  transform: translate(-12px, 12px);
  z-index: 3;
}

button.btn.striped-shadow:hover:after,
button.btn.striped-shadow:hover:before {
  -webkit-transform: translate(12px, -12px);
  -ms-transform: translate(12px, -12px);
  transform: translate(12px, -12px);
}
button.btn.striped-shadow:before {
  width: 8px;
  max-height: calc(100% - 5px);
  height: 100%;
  left: -12px;
  bottom: -5px;
  background-size: 8px 15px;
  background-repeat: repeat-y;
  background-position: 0 100%;
}

.input {
  margin: 5% 10%;
  position: relative;
  width: fit-content;
}
input {
  padding: 10px 10px 10px 5px;
  font-size: 18px;
  width: 280px;
  border: 1px solid;
  border-color: transparent transparent gray;
  background-color: transparent;
}
input:focus {
  outline: none;
}
/*Label */
label {
  position: absolute;
  top: 30%;
  font-size: 18px;
  color: rgb(165, 165, 165);
  left: 3%;
  z-index: -1;
  pointer-events: none;
  transition: all 0.3s;
  -webkit-transition: all 0.3s;
  -moz-transition: all 0.3s;
  -ms-transition: all 0.3s;
  -o-transition: all 0.3s;
}
/* Activate State */
input:focus + label,
input:valid + label {
  font-size: 12px;
  color: rgb(148, 98, 255);
  top: -1%;
  transition: all 0.3s;
  -webkit-transition: all 0.3s;
  -moz-transition: all 0.3s;
  -ms-transition: all 0.3s;
  -o-transition: all 0.3s;
}
/*End Label */
/*Bar*/
.bar {
  width: 100%;
  height: 2px;
  position: absolute;
  background-color: rgb(148, 98, 255);
  top: calc(100% - 2px);
  left: 0;
  transform: scaleX(0);
  -webkit-transform: scaleX(0);
  -moz-transform: scaleX(0);
  -ms-transform: scaleX(0);
  -o-transform: scaleX(0);
}
/*Activate State */
input:focus ~ .bar,
input:valid ~ .bar {
  transform: scaleX(1);
  -webkit-transform: scaleX(1);
  -moz-transform: scaleX(1);
  -ms-transform: scaleX(1);
  -o-transform: scaleX(1);
  transition: transform 0.3s;
  -webkit-transition: transform 0.3s;
  -moz-transition: transform 0.3s;
  -ms-transition: transform 0.3s;
  -o-transition: transform 0.3s;
}
/*End Bar */
/*Highlight */
.highlight {
  width: 100%;
  height: 85%;
  position: absolute;
  background-color: rgba(148, 98, 255, 0.2);
  top: 15%;
  left: 0;
  visibility: hidden;
  z-index: -1;
}
input:focus ~ .highlight {
  width: 0;
  visibility: visible;
  transition: all 0.09s linear;
  -webkit-transition: all 0.09s linear;
  -moz-transition: all 0.09s linear;
  -ms-transition: all 0.09s linear;
  -o-transition: all 0.09s linear;
}
/*End highlight */
::placeholder {
  color: #77bfa1;
  font-size: 20px;
}
</style>

</html>"""
h = HTML(s)
display(h)

In [0]:
"""
!pip install widgetsnbextension
!pip install ipywidgets
!pip install voila
from IPython.display import HTML

import ipywidgets as widgets
from IPython.display import display, clear_output
!jupyter nbextension enable --py widgetsnbextension --sys-prefix
!jupyter serverextension enable --sys-prefix

button_send = widgets.Button(description='Check',tooltip='Check')
output = widgets.Output()
def on_button_clicked(event):
  with output:
    clear_output()
    print("Sent message")
button_send.on_click(on_button_clicked)
vbox_result = widgets.VBox([button_send, output])
text_0 = widgets.HTML(value='<h1>Hey</h1>')
vbox_text = widgets.VBox([text_0])
page = widgets.HBox([vbox_text])
display(page)
"""

## Check everything is ok

In [0]:
spark
sc._conf.getAll()

Out[19]: [('spark.files.useFetchCache', 'false'),
 ('spark.databricks.preemption.enabled', 'true'),
 ('spark.driver.tempDirectory', '/local_disk0/tmp'),
 ('spark.hadoop.fs.adl.impl.disable.cache', 'true'),
 ('spark.hadoop.parquet.block.size.row.check.max', '10'),
 ('spark.hadoop.fs.s3a.connection.maximum', '200'),
 ('spark.databricks.clusterUsageTags.numPerClusterInitScriptsV2', '0'),
 ('spark.executor.extraJavaOptions',
 '-Djava.io.tmpdir=/local_disk0/tmp -XX:ReservedCodeCacheSize=512m -XX:+UseCodeCacheFlushing -Djava.security.properties=/databricks/spark/dbconf/java/extra.security -XX:-UseContainerSupport -XX:+PrintFlagsFinal -XX:+PrintGCDateStamps -verbose:gc -XX:+PrintGCDetails -Xss4m -Djava.library.path=/usr/java/packages/lib/amd64:/usr/lib64:/lib64:/lib:/usr/lib:/usr/lib/x86_64-linux-gnu/jni:/lib/x86_64-linux-gnu:/usr/lib/x86_64-linux-gnu:/usr/lib/jni -Djavax.xml.datatype.DatatypeFactory=com.sun.org.apache.xerces.internal.jaxp.datatype.DatatypeFactoryImpl -Djavax.xml.parsers.DocumentBuilderFactory=com.sun.org.apache.xerces.internal.jaxp.DocumentBuilderFactoryImpl -Djavax.xml.parsers.SAXParserFactory=com.sun.org.apache.xerces.internal.jaxp.SAXParserFactoryImpl -Djavax.xml.validation.SchemaFactory:http://www.w3.org/2001/XMLSchema=com.sun.org.apache.xerces.internal.jaxp.validation.XMLSchemaFactory -Dorg.xml.sax.driver=com.sun.org.apache.xerces.internal.parsers.SAXParser -Dorg.w3c.dom.DOMImplementationSourceList=com.sun.org.apache.xerces.internal.dom.DOMXSImplementationSourceImpl -Djavax.net.ssl.sessionCacheSize=10000 -Dscala.reflect.runtime.disable.typetag.cache=true -Ddatabricks.serviceName=spark-executor-1'),
 ('spark.hadoop.fs.s3a.fast.upload.active.blocks', '32'),
 ('spark.shuffle.reduceLocality.enabled', 'false'),
 ('spark.sql.streaming.checkpointFileManagerClass',
 'com.databricks.spark.sql.streaming.DatabricksCheckpointFileManager'),
 ('spark.databricks.service.dbutils.repl.backend',
 'com.databricks.dbconnect.ReplDBUtils'),
 ('spark.databricks.clusterUsageTags.driverNodeType', 'dev-tier-node'),
 ('spark.hadoop.spark.sql.sources.outputCommitterClass',
 'com.databricks.backend.daemon.data.client.MapReduceDirectOutputCommitter'),
 ('spark.hadoop.fs.AbstractFileSystem.gs.impl',
 'shaded.databricks.V2_1_4.com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS'),
 ('spark.databricks.clusterUsageTags.instanceBootstrapType', 'ssh'),
 ('spark.streaming.driver.writeAheadLog.allowBatching', 'true'),
 ('spark.ui.port', '42506'),
 ('spark.databricks.clusterSource', 'UI'),
 ('spark.hadoop.hive.server2.transport.mode', 'http'),
 ('spark.hadoop.hive.server2.thrift.http.cookie.auth.enabled', 'false'),
 ('spark.executor.memory', '8278m'),
 ('spark.databricks.driverNodeTypeId', 'dev-tier-node'),
 ('spark.sql.parquet.compression.codec', 'snappy'),
 ('spark.hadoop.fs.cpfs-adl.impl.disable.cache', 'true'),
 ('spark.databricks.clusterUsageTags.hailEnabled', 'false'),
 ('spark.databricks.clusterUsageTags.clusterLogDeliveryEnabled', 'false'),
 ('spark.databricks.clusterUsageTags.containerType', 'LXC'),
 ('spark.eventLog.enabled', 'false'),
 ('spark.databricks.clusterUsageTags.isIMv2Enabled', 'false'),
 ('spark.databricks.cloudfetch.hasRegionSupport', 'true'),
 ('spark.hadoop.databricks.s3.create.deleteUnnecessaryFakeDirectories',
 'false'),
 ('spark.hadoop.fs.wasb.impl',
 'shaded.databricks.org.apache.hadoop.fs.azure.NativeAzureFileSystem'),
 ('spark.executor.tempDirectory', '/local_disk0/tmp'),
 ('spark.databricks.workerNodeTypeId', 'dev-tier-node'),
 ('spark.hadoop.mapred.output.committer.class',
 'com.databricks.backend.daemon.data.client.DirectOutputCommitter'),
 ('spark.hadoop.hive.server2.thrift.http.port', '10000'),
 ('spark.hadoop.mapreduce.fileoutputcommitter.algorithm.version', '2'),
 ('spark.sql.allowMultipleContexts', 'false'),
 ('spark.databricks.clusterUsageTags.clusterEbsVolumeSize', '0'),
 ('spark.home', '/databricks/spark'),
 ('spark.databricks.clusterUsageTags.clusterTargetWorkers', '0'),
 ('spark.sql.warehouse.dir', '/user/hive/wa

## Upload Dataset

In [0]:
bot_df = spark.read.format("csv").load("dbfs:/FileStore/shared_uploads/tavoulari.1977701@studenti.uniroma1.it/twitter_human_bots_dataset_prt1.csv",header='true')

## Convert numbers stored as text to numbers

In [0]:
bot_df_text = bot_df

# Drop duplicates
bot_df.dropDuplicates(["id"])

# Subtract profiles' creation dates from current date to convert this field into pure integers
def to_days(then):
  now = datetime.datetime.now()
  date_time_obj = datetime.datetime.strptime(then, '%Y-%m-%d %H:%M:%S').date()
  diff =(now.date() - date_time_obj)
  diff = str(diff).split(' ')
  return int(diff[0])

to_days_UDF = spark.udf.register("to_days",to_days)
bot_df = bot_df.withColumn("created_at", to_days_UDF(col("created_at")))

# Cast numerical features from string to int/float
bot_df = bot_df.selectExpr("account_type","cast(follower_count as int) follower_count","cast(friends_count as int) friends_count","cast(listed_count as int) listed_count","cast(statuses_count as int) statuses_count","cast(retweets as float) retweets","cast(with_url as float) with_url","cast(with_mention as float) with_mention","geo_enabled", "verified", "has_extended_profile", "default_profile", "default_profile_image","cast(created_at as int) created_at","cast(avg_cosine as float) avg_cosine")

# Textual fields
bot_df_text = bot_df_text.selectExpr("account_type", "description", "tweet_text")
bot_df.printSchema()
bot_df.show(5)


root
-- account_type: string (nullable = true)
-- follower_count: integer (nullable = true)
-- friends_count: integer (nullable = true)
-- listed_count: integer (nullable = true)
-- statuses_count: integer (nullable = true)
-- retweets: float (nullable = true)
-- with_url: float (nullable = true)
-- with_mention: float (nullable = true)
-- geo_enabled: string (nullable = true)
-- verified: string (nullable = true)
-- has_extended_profile: string (nullable = true)
-- default_profile: string (nullable = true)
-- default_profile_image: string (nullable = true)
-- created_at: integer (nullable = true)
-- avg_cosine: float (nullable = true)

+------------+--------------+-------------+------------+--------------+----------+----------+------------+-----------+--------+--------------------+---------------+---------------------+----------+----------+
account_type|follower_count|friends_count|listed_count|statuses_count| retweets| with_url|with_mention|geo_enabled|verified|has_extended_profile|default_profile|default_profile_image|created_at|avg_cosine|
+------------+--------------+-------------+------------+--------------+----------+----------+------------+-----------+--------+--------------------+---------------+---------------------+----------+----------+
 bot| 1707| 4| 53| 13180| 0.0| 0.0| 0.0| False| False| False| False| False| 1675| 0.2399768|
 human| 821| 850| 5| 251|0.34108528|0.33333334| 0.4031008| False| False| True| False| False| 1650|0.13450052|
 human| 194| 808| 2| 1208|0.15384616|0.18461539| 0.26923078| True| False| False| False| False| 1430|0.09770669|
 human| 572| 662| 2| 1661|0.63846153|0.06923077| 0.9153846| True| False| False| True| False| 1761|0.07793919|
 human| 747532| 118| 1817| 4296|0.23255815| 0.2015504| 0.74418604| True| True| False| False| False| 3410|0.06907502|
+------------+--------------+-------------+------------+--------------+----------+----------+------------+-----------+--------+--------------------+---------------+---------------------+----------+----------+
only showing top 5 rows

## Split features into categories

In [0]:
NUMERICAL_FEATURES = ["follower_count", 
                      "friends_count",
                      "listed_count",
                      "statuses_count",
                      "retweets",
                      "with_url",
                      "with_mention",
                      "created_at",
                      "avg_cosine"
                      ]
CATEGORICAL_FEATURES = ["geo_enabled", 
                        "verified", 
                        "has_extended_profile",
                        "default_profile",
                        "default_profile_image",
                        ]

TEXTUAL_FEATURES =  ["description",
                     "tweet_text"
                     ]

TARGET_VARIABLE = "account_type"

In [0]:
bot_df.groupBy(TARGET_VARIABLE).count().show()
train_df, test_df = bot_df.randomSplit([0.8, 0.2], seed=RANDOM_SEED)

+------------+-----+
account_type|count|
+------------+-----+
 human| 9822|
 bot| 5036|
+------------+-----+

## Vectorization - Logistic Regression - Tuning Hyperparameters

In [0]:
# This function defines the general pipeline for logistic regression
def logistic_regression_pipeline(train, 
                                 numerical_features, 
                                 categorical_features, 
                                 target_variable, 
                                 with_std=True,
                                 with_mean=True,
                                 k_fold=5):

    # 1.a Create a list of indexers, i.e., one for each categorical feature
    indexers = [StringIndexer(inputCol=c, outputCol="{0}_indexed".format(c), handleInvalid="keep") for c in categorical_features]

    # 1.b Create the one-hot encoder for the list of features just indexed (this encoder will keep any unseen label in the future)
    encoder = OneHotEncoder(inputCols=[indexer.getOutputCol() for indexer in indexers], 
                                    outputCols=["{0}_encoded".format(indexer.getOutputCol()) for indexer in indexers], 
                                    handleInvalid="keep")

    # 1.c Indexing the target column (i.e., transform human/bot into 0/1) and rename it as "label"
    label_indexer = StringIndexer(inputCol = target_variable, outputCol = "label")
    
    # 1.d Assemble all the features (both one-hot-encoded categorical and numerical) into a single vector
    assembler = VectorAssembler(inputCols=encoder.getOutputCols() + numerical_features, outputCol="features")

    # 2.a Create the StandardScaler
    scaler = StandardScaler(inputCol=assembler.getOutputCol(), outputCol="std_"+assembler.getOutputCol(), withStd=with_std, withMean=with_mean)
    # ...

    # 3 Populate the stages of the pipeline with all the preprocessing steps
    stages = indexers + [encoder] + [label_indexer] + [assembler]  + [scaler] #+ ...

    # 4. Create the logistic regression transformer
    log_reg = LogisticRegression(featuresCol="std_features", labelCol="label", maxIter=100) # change `featuresCol=std_features` if scaler is used
    # 5. Add the logistic regression transformer to the pipeline stages (i.e., the last one)
    stages += [log_reg]

    # 6. Set up the pipeline
    pipeline = Pipeline(stages=stages)

    # We use a ParamGridBuilder to construct a grid of parameters to search over.
    # A CrossValidator requires an Estimator, a set of Estimator ParamMaps, and an Evaluator.
    # We use a ParamGridBuilder to construct a grid of parameters to search over.
    # With 3 values for log_reg.regParam ($\lambda$) and 3 values for log_reg.elasticNetParam ($\alpha$),
    # this grid will have 3 x 3 = 9 parameter settings for CrossValidator to choose from.
    param_grid = ParamGridBuilder()\
    .addGrid(log_reg.regParam, [0.0, 0.05, 0.1]) \
    .addGrid(log_reg.elasticNetParam, [0.0, 0.5, 1.0]) \
    .build()
    
    cross_val = CrossValidator(estimator=pipeline, 
                               estimatorParamMaps=param_grid,
                               evaluator=BinaryClassificationEvaluator(metricName="areaUnderROC"), # default = "areaUnderROC", alternatively "areaUnderPR"
                               numFolds=k_fold,
                               collectSubModels=True # this flag allows us to store ALL the models trained during k-fold cross validation
                               )

    # Run cross-validation, and choose the best set of parameters.
    cv_model = cross_val.fit(train)

    return cv_model

### Training Set

In [0]:
cv_model = logistic_regression_pipeline(train_df, NUMERICAL_FEATURES, CATEGORICAL_FEATURES, TARGET_VARIABLE)

In [0]:
# This function summarizes all the models trained during k-fold cross validation
def summarize_all_models(cv_models):
    for k, models in enumerate(cv_models):
        print("*************** Fold #{:d} ***************\n".format(k+1))
        for i, m in enumerate(models):
            print("--- Model #{:d} out of {:d} ---".format(i+1, len(models)))
            print("\tParameters: lambda=[{:.3f}]; alpha=[{:.3f}] ".format(m.stages[-1]._java_obj.getRegParam(), m.stages[-1]._java_obj.getElasticNetParam()))
            print("\tModel summary: {}\n".format(m.stages[-1]))
        print("***************************************\n")

In [0]:
# Call the function above
summarize_all_models(cv_model.subModels)

In [0]:
for i, avg_roc_auc in enumerate(cv_model.avgMetrics):
    print("Avg. ROC AUC computed across k-fold cross validation for model setting #{:d}: {:.3f}".format(i+1, avg_roc_auc))

In [0]:
print("Best model according to k-fold cross validation: lambda=[{:.3f}]; alfa=[{:.3f}]".
      format(cv_model.bestModel.stages[-1]._java_obj.getRegParam(), 
             cv_model.bestModel.stages[-1]._java_obj.getElasticNetParam(),
             )
      )
print(cv_model.bestModel.stages[-1])

In [0]:
# `bestModel` is the best resulting model according to k-fold cross validation, which is also entirely retrained on the whole `train_df`
training_result = cv_model.bestModel.stages[-1].summary

### Test Set

In [0]:
# Make predictions on the test set (`cv_model` contains the best model according to the result of k-fold cross validation)
# `test_df` will follow exactly the same pipeline defined above, and already fit to `train_df`
test_predictions = cv_model.transform(test_df)

In [0]:
def evaluate_model(predictions, metric="areaUnderROC"):
    evaluator = BinaryClassificationEvaluator(metricName=metric)
    return evaluator.evaluate(predictions)

### Evaluation

In [0]:
print("***** Training Set *****")
print("Area Under ROC Curve (ROC AUC): {:.3f}".format(training_result.areaUnderROC))


plt.figure(figsize=(5,5))
# roc 
#plt.subplot(2, 6, 1)
plt.plot([0, 1], [0, 1], 'r--')
plt.plot(training_result.roc.select('FPR').collect(),
         training_result.roc.select('TPR').collect())
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.show()

#plt.subplot(2, 6, 2)
plt.plot([0, 1], [0, 1], 'r--')
plt.plot(training_result.pr.select('recall').collect(),
         training_result.pr.select('precision').collect())
plt.xlabel('recall')
plt.ylabel('presicion')
plt.show()

#plt.subplot(2, 6, 3)
plt.plot([0, 1], [0, 1], 'r--')
plt.plot(training_result.precisionByThreshold.select('threshold').collect(),
         training_result.precisionByThreshold.select('precision').collect())
plt.xlabel('threshold')
plt.ylabel('precision')
plt.show()


#plt.subplot(2, 6, 4)
plt.plot([0, 1], [0, 1], 'r--')
plt.plot(training_result.recallByThreshold.select('threshold').collect(),
         training_result.recallByThreshold.select('recall').collect())
plt.xlabel('threshold')
plt.ylabel('recall')
plt.show()

#plt.subplot(2, 6, 5)
plt.plot([0, 1], [0, 1], 'r--')
plt.plot(training_result.fMeasureByThreshold.select('threshold').collect(),
         training_result.fMeasureByThreshold.select('F-Measure').collect())
plt.xlabel('threshold')
plt.ylabel('F-Measure')


plt.show()

print("***** Test Set *****")
print("Area Under ROC Curve (ROC AUC): {:.3f}".format(evaluate_model(test_predictions)))
print("Area Under Precision-Recall Curve: {:.3f}".format(evaluate_model(test_predictions, metric="areaUnderPR")))

## Desicion Tree

In [0]:
# This function defines the general pipeline for logistic regression
def decision_tree_pipeline(train, 
                           numerical_features, 
                           categorical_features, 
                           target_variable, 
                           with_std=True,
                           with_mean=True,
                           k_fold=5):

    indexers = [StringIndexer(inputCol=c, outputCol="{0}_indexed".format(c), handleInvalid="keep") for c in categorical_features]

    # Indexing the target column (i.e., transform human/bot into 0/1) and rename it as "label"
    label_indexer = StringIndexer(inputCol = target_variable, outputCol = "label")
    
    # Assemble all the features (both one-hot-encoded categorical and numerical) into a single vector
    assembler = VectorAssembler(inputCols=[indexer.getOutputCol() for indexer in indexers] + numerical_features, outputCol="features")

    # Populate the stages of the pipeline with all the preprocessing steps
    stages = indexers + [label_indexer] + [assembler] # + ...

    # Create the decision tree transformer
    dt = DecisionTreeClassifier(featuresCol="features", labelCol="label") # change `featuresCol=std_features` if scaler is used

    # 5. Add the decision tree transformer to the pipeline stages (i.e., the last one)
    stages += [dt]

    # 6. Set up the pipeline
    pipeline = Pipeline(stages=stages)

    # We use a ParamGridBuilder to construct a grid of parameters to search over.
    # A CrossValidator requires an Estimator, a set of Estimator ParamMaps, and an Evaluator.
    # We use a ParamGridBuilder to construct a grid of parameters to search over.
    # With 3 values for dt.maxDepth and 2 values for dt.impurity
    # this grid will have 3 x 2 = 9 parameter settings for CrossValidator to choose from.
    param_grid = ParamGridBuilder()\
    .addGrid(dt.maxDepth, [3, 5, 8]) \
    .addGrid(dt.impurity, ["gini", "entropy"]) \
    .build()
    
    cross_val = CrossValidator(estimator=pipeline, 
                               estimatorParamMaps=param_grid,
                               evaluator=BinaryClassificationEvaluator(metricName="areaUnderROC"), # default = "areaUnderROC", alternatively "areaUnderPR"
                               numFolds=k_fold,
                               collectSubModels=True # this flag allows us to store ALL the models trained during k-fold cross validation
                               )

    # Run cross-validation, and choose the best set of parameters.
    cv_model = cross_val.fit(train)

    return cv_model

### Train Set

In [0]:
cv_model = decision_tree_pipeline(train_df, NUMERICAL_FEATURES, CATEGORICAL_FEATURES, TARGET_VARIABLE)

In [0]:
# This function summarizes all the models trained during k-fold cross validation

def summarize_all_models(cv_models):
    for k, models in enumerate(cv_models):
        print("*************** Fold #{:d} ***************\n".format(k+1))
        for i, m in enumerate(models):
            print("--- Model #{:d} out of {:d} ---".format(i+1, len(models)))
            print("\tParameters: maxDept=[{:d}]; impurity=[{:s}] ".format(m.stages[-1]._java_obj.getMaxDepth(), m.stages[-1]._java_obj.getImpurity()))
            print("\tModel summary: {}\n".format(m.stages[-1]))
        print("***************************************\n")

In [0]:
summarize_all_models(cv_model.subModels)

In [0]:
training_result = 0
for i, avg_roc_auc in enumerate(cv_model.avgMetrics):
    print("Avg. ROC AUC computed across k-fold cross validation for model setting #{:d}: {:.3f}".format(i+1, avg_roc_auc))
    if training_result < avg_roc_auc:
      training_result = avg_roc_auc

In [0]:
print("Best model according to k-fold cross validation: maxDept=[{:d}]; impurity=[{:s}]".
      format(cv_model.bestModel.stages[-1]._java_obj.getMaxDepth(), 
             cv_model.bestModel.stages[-1]._java_obj.getImpurity(),
             )
      )
print(cv_model.bestModel.stages[-1])

### Test Set

In [0]:
# Make predictions on the test set (`cv_model` contains the best model according to the result of k-fold cross validation)
# `test_df` will follow exactly the same pipeline defined above, and already fit to `train_df`
test_predictions = cv_model.transform(test_df)
test_predictions.select("features", "prediction", "label").show(5)

### Evaluation

In [0]:
print("***** Training Set *****")
print("Area Under ROC Curve (ROC AUC): {:.3f}".format(training_result))


print("***** Test Set *****")
print("Area Under ROC Curve (ROC AUC): {:.3f}".format(evaluate_model(test_predictions)))
print("Area Under Precision-Recall Curve: {:.3f}".format(evaluate_model(test_predictions, metric="areaUnderPR")))

## Random Forests

In [0]:
# This function defines the general pipeline for logistic regression
def random_forest_pipeline(train, 
                           numerical_features, 
                           categorical_features, 
                           target_variable, 
                           with_std=True,
                           with_mean=True,
                           k_fold=5):


    # Configure a random forest pipeline, which consists of the following stages: 

    indexers = [StringIndexer(inputCol=c, outputCol="{0}_indexed".format(c), handleInvalid="keep") for c in categorical_features]

    # Indexing the target column (i.e., transform human/bot into 0/1) and rename it as "label"
    label_indexer = StringIndexer(inputCol = target_variable, outputCol = "label")
    
    # Assemble all the features (both one-hot-encoded categorical and numerical) into a single vector
    assembler = VectorAssembler(inputCols=[indexer.getOutputCol() for indexer in indexers] + numerical_features, outputCol="features")

    # Populate the stages of the pipeline with all the preprocessing steps
    stages = indexers + [label_indexer] + [assembler] # + ...

    # Create the random forest transformer
    rf = RandomForestClassifier(featuresCol="features", labelCol="label") # change `featuresCol=std_features` if scaler is used

    # 5. Add the random forest transformer to the pipeline stages (i.e., the last one)
    stages += [rf]

    # 6. Set up the pipeline
    pipeline = Pipeline(stages=stages)

    # We use a ParamGridBuilder to construct a grid of parameters to search over.
    # A CrossValidator requires an Estimator, a set of Estimator ParamMaps, and an Evaluator.
    # We use a ParamGridBuilder to construct a grid of parameters to search over.
    # With 3 values for rf.maxDepth and 3 values for rf.numTrees
    # this grid will have 3 x 3 = 9 parameter settings for CrossValidator to choose from.
    param_grid = ParamGridBuilder()\
    .addGrid(rf.maxDepth, [3, 5, 8]) \
    .addGrid(rf.numTrees, [10, 50, 100]) \
    .build()
    cross_val = CrossValidator(estimator=pipeline, 
                               estimatorParamMaps=param_grid,
                               evaluator=BinaryClassificationEvaluator(metricName="areaUnderROC"), # default = "areaUnderROC", alternatively "areaUnderPR"
                               numFolds=k_fold,
                               collectSubModels=True # this flag allows us to store ALL the models trained during k-fold cross validation
                               )

    # Run cross-validation, and choose the best set of parameters.
    cv_model = cross_val.fit(train)

    return cv_model


In [0]:
cv_model = random_forest_pipeline(train_df, NUMERICAL_FEATURES, CATEGORICAL_FEATURES, TARGET_VARIABLE)
training_result = 0
for i, avg_roc_auc in enumerate(cv_model.avgMetrics):
    print("Avg. ROC AUC computed across k-fold cross validation for model setting #{:d}: {:.3f}".format(i+1, avg_roc_auc))
    if avg_roc_auc > training_result:
      training_result = avg_roc_auc
    
print("Best model according to k-fold cross validation: maxDept=[{:d}]".
      format(cv_model.bestModel.stages[-1]._java_obj.getMaxDepth(),),)

print(cv_model.bestModel.stages[-1])

In [0]:
#training_result = cv_model.bestModel.stages[-1].summary
# Make predictions on the test set (`cv_model` contains the best model according to the result of k-fold cross validation)
# `test_df` will follow exactly the same pipeline defined above, and already fit to `train_df`
test_predictions = cv_model.transform(test_df)
test_predictions.select("features", "prediction", "label").show(5)

In [0]:
print("***** Training Set *****")
print("Area Under ROC Curve (ROC AUC): {:.3f}".format(training_result))


print("***** Test Set *****")
print("Area Under ROC Curve (ROC AUC): {:.3f}".format(evaluate_model(test_predictions)))
print("Area Under Precision-Recall Curve: {:.3f}".format(evaluate_model(test_predictions, metric="areaUnderPR")))

## Factorization Machines

In [0]:
def fm_pipeline(train, 
                numerical_features,
                categorical_features, 
                target_variable, 
                with_std=True,
                with_mean=True,
                k_fold=5):



    # 1.a Create a list of indexers, i.e., one for each categorical feature
    indexers = [StringIndexer(inputCol=c, outputCol="{0}_indexed".format(c), handleInvalid="keep") for c in categorical_features]

    # 1.b Create the one-hot encoder for the list of features just indexed (this encoder will keep any unseen label in the future)
    encoder = OneHotEncoder(inputCols=[indexer.getOutputCol() for indexer in indexers], 
                                    outputCols=["{0}_encoded".format(indexer.getOutputCol()) for indexer in indexers], 
                                    handleInvalid="keep")


    # Indexing the target column (i.e., transform human/bot into 0/1) and rename it as "label"
    label_indexer = StringIndexer(inputCol = target_variable, outputCol = "label")
    
    # Assemble all the features (both one-hot-encoded categorical and numerical) into a single vector
    assembler = VectorAssembler(inputCols=[indexer.getOutputCol() for indexer in indexers] + numerical_features, outputCol="features")

    featureScaler = MinMaxScaler(inputCol="features", outputCol="scaledFeatures")
    
    stages = indexers + [encoder] + [label_indexer] + [assembler] + [featureScaler]
    
    fm = FMClassifier(labelCol="label", featuresCol="scaledFeatures")

    # 5. Add the decision tree transformer to the pipeline stages (i.e., the last one)
    stages += [fm]

    # 6. Set up the pipeline
    pipeline = Pipeline(stages=stages)

    # We use a ParamGridBuilder to construct a grid of parameters to search over.
    # A CrossValidator requires an Estimator, a set of Estimator ParamMaps, and an Evaluator.
    # We use a ParamGridBuilder to construct a grid of parameters to search over.
    # this grid will have 3 x 3 = 9 parameter settings for CrossValidator to choose from.
    param_grid = ParamGridBuilder()\
    .addGrid(fm.stepSize, [0.001,0.002,0.005,0.01]) \
    .build()
    
    cross_val = CrossValidator(estimator=pipeline, 
                               estimatorParamMaps=param_grid,
                               evaluator=BinaryClassificationEvaluator(metricName="areaUnderROC"), # default = "areaUnderROC", alternatively "areaUnderPR"
                               numFolds=k_fold,
                               collectSubModels=True # this flag allows us to store ALL the models trained during k-fold cross validation
                               )

    # Run cross-validation, and choose the best set of parameters.
    cv_model = cross_val.fit(train)

    return cv_model

In [0]:
cv_model = fm_pipeline(train_df, NUMERICAL_FEATURES, CATEGORICAL_FEATURES, TARGET_VARIABLE)

In [0]:
training_result = 0
for i, avg_roc_auc in enumerate(cv_model.avgMetrics):
    print("Avg. ROC AUC computed across k-fold cross validation for model setting #{:d}: {:.3f}".format(i+1, avg_roc_auc))
    if avg_roc_auc > training_result:
      training_result = avg_roc_auc

In [0]:
#training_result = cv_model.bestModel.stages[-1].summary
# Make predictions on the test set (`cv_model` contains the best model according to the result of k-fold cross validation)
# `test_df` will follow exactly the same pipeline defined above, and already fit to `train_df`
test_predictions = cv_model.transform(test_df)
test_predictions.select("features", "prediction", "label").show(5)

In [0]:
print("***** Training Set *****")
print("Area Under ROC Curve (ROC AUC): {:.3f}".format(training_result))


print("***** Test Set *****")
print("Area Under ROC Curve (ROC AUC): {:.3f}".format(evaluate_model(test_predictions)))
print("Area Under Precision-Recall Curve: {:.3f}".format(evaluate_model(test_predictions, metric="areaUnderPR")))

## Gradient Boosted Decision Tree

In [0]:
# This function defines the general pipeline for logistic regression
def gbdt_pipeline(train, 
                  numerical_features, 
                  categorical_features, 
                  target_variable, 
                  with_std=True,
                  with_mean=True,
                  k_fold=5):

    indexers = [StringIndexer(inputCol=c, outputCol="{0}_indexed".format(c), handleInvalid="keep") for c in categorical_features]

    # Indexing the target column (i.e., transform human/bot into 0/1) and rename it as "label"
    label_indexer = StringIndexer(inputCol = target_variable, outputCol = "label")
    
    # Assemble all the features (both one-hot-encoded categorical and numerical) into a single vector
    assembler = VectorAssembler(inputCols=[indexer.getOutputCol() for indexer in indexers] + numerical_features, outputCol="features")

    # Populate the stages of the pipeline with all the preprocessing steps
    stages = indexers + [label_indexer] + [assembler] # + ...

    # Create the gradient boosted decision tree transformer
    gbdt = GBTClassifier(featuresCol="features", labelCol="label") # change `featuresCol=std_features` if scaler is used

    # Add the gradient boosted decision tree transformer to the pipeline stages (i.e., the last one)
    stages += [gbdt]

    # 6. Set up the pipeline
    pipeline = Pipeline(stages=stages)

    # We use a ParamGridBuilder to construct a grid of parameters to search over.
    # A CrossValidator requires an Estimator, a set of Estimator ParamMaps, and an Evaluator.
    # We use a ParamGridBuilder to construct a grid of parameters to search over.
    # With 3 values for gbdt.maxDepth and 3 values for gbdt.maxIter (i.e., boosting rounds)
    # this grid will have 3 x 3 = 9 parameter settings for CrossValidator to choose from.
    param_grid = ParamGridBuilder()\
    .addGrid(gbdt.maxDepth, [3, 5, 8]) \
    .addGrid(gbdt.maxIter, [10, 50, 100]) \
    .build()
    
    cross_val = CrossValidator(estimator=pipeline, 
                               estimatorParamMaps=param_grid,
                               evaluator=BinaryClassificationEvaluator(metricName="areaUnderROC"), # default = "areaUnderROC", alternatively "areaUnderPR"
                               numFolds=k_fold,
                               collectSubModels=True # this flag allows us to store ALL the models trained during k-fold cross validation
                               )
    
    # Run cross-validation, and choose the best set of parameters.
    cv_model = cross_val.fit(train)

    return cv_model
cv_model = gbdt_pipeline(train_df, NUMERICAL_FEATURES, CATEGORICAL_FEATURES, TARGET_VARIABLE)


In [0]:
for i, avg_roc_auc in enumerate(cv_model.avgMetrics):
    print("Avg. ROC AUC computed across k-fold cross validation for model setting #{:d}: {:.3f}".format(i+1, avg_roc_auc))
print("Best model according to k-fold cross validation: maxDept=[{:d}]; maxIter=[{:d}]".
      format(cv_model.bestModel.stages[-1]._java_obj.getMaxDepth(), 
             cv_model.bestModel.stages[-1]._java_obj.getMaxIter()
             )
      )
print(cv_model.bestModel.stages[-1])
# Make predictions on the test set (`cv_model` contains the best model according to the result of k-fold cross validation)
# `test_df` will follow exactly the same pipeline defined above, and already fit to `train_df`
test_predictions = cv_model.transform(test_df)
print("***** Test Set *****")
print("Area Under ROC Curve (ROC AUC): {:.3f}".format(evaluate_model(test_predictions)))
print("Area Under Precision-Recall Curve: {:.3f}".format(evaluate_model(test_predictions, metric="areaUnderPR")))
print("***** Test Set *****")

## Logistic Regression on textual features

## Clean text

In [0]:
def clean_text(text):

  if text is None:
    return ""
  if text == []:
    return ""
  row = text.lower()
  row = row.strip() 
  row = re.sub(r'[^\w\s]',' ',row)
  row = re.sub(r'\_',' ',row)

  filtered_sentence = ""
  for w in row.split() :
    temp = porter.stem(w)
    filtered_sentence += (temp + " ")
  row = filtered_sentence
  if row is None:
    return ""
  if text == []:
    return ""
  return row
clean_udf = spark.udf.register("clean_text",clean_text)

### Train-Test Split

In [0]:
bot_df_text = bot_df_text.select(clean_udf(col("tweet_text")) , clean_udf(col("description")),"account_type")
tweet_train_df, tweet_test_df = bot_df_text.randomSplit([0.8, 0.2], seed=RANDOM_SEED)

tweet_train_df = tweet_train_df.withColumnRenamed("clean_text(tweet_text)", "tweet_text")\
       .withColumnRenamed("clean_text(description)", "description")

In [0]:
tweet_test_df.show(1000)

+----------------------+-----------------------+------------+
clean_text(tweet_text)|clean_text(description)|account_type|
+----------------------+-----------------------+------------+
 | | bot|
 | | bot|
 | | bot|
 | | bot|
 | | bot|
 | | bot|
 | | bot|
 | | bot|
 | | bot|
 | | bot|
 | | bot|
 | | bot|
 | | bot|
 | | bot|
 | | bot|
 | | bot|
 | | bot|
 | | bot|
 | | bot|
 | | bot|
 | | bot|
 | | bot|
 | | bot|
 | | bot|
 | | bot|
 | | bot|
 | | bot|
 | | bot|
 | | bot|
 | | human|
 | | human|
 | a bot also on mas...| bot|
 | ahmo wh | bot|
 | ashanti siddik ha...| human|
 | begin are usual s...| bot|
 | day trade invest ...| bot|
 | fish work and fish | bot|
 | fuck whoever tell...| bot|
 | hakszlk karsnda s...| human|
 | hatter to the whi...| bot|
 | hi my name is don...| human|
 | hodl | bot|
 | ive been a fan si...| bot|
 | jordan s law an i...| bot|
 | junior | bot|
 | knead we say more | bot|
 | la fortuna nn esi...| human|
 | need your song to...| bot|
 | never look for th...| bot|
 | peridico digit ve...| human|
 | senadora de la re...| human|
 | setiap tweet nase...| human|
 | snapchat edgar se...| human|
 | statu page da fox...| bot|
 | vicdann senin kbl...| human|
 | wtf | bot|
 11 30 kbs1 lt gt ...| | human|
 32 cowok nonton f...| no one live forev | bot|
 a lili veloso arr...| pri schiavinato a...| human|
 a true legend rip...| contentstrategi s...| human|
 after the ice sto...| award win and emm...| human|
 aiai mulher t co ...| the bar is go to ...| bot|
 all eui process p...| news from the eur...| human|
 amsh75 f99i amsh7...| | human|
 and i am so sorri...| de motiv speaker | human|
 and moy noym auys...| she her 22 real l...| human|
 anthianthia spdap...| event and adverti...| bot|
 applesupport my m...| 26 she her must g...| human|
 arac for lovexara...| arafamilia slo ha...| human|
 arm forc day may ...| peopl help peopl ...| human|
 as you may have n...| singer songwrit m...| human|
 awhspe t co vc8iu...| chemist postdoc i...| human|
 b t co iwtn8sermk...| nhk b over the wave | human|
 be down with covi...| life is about mak...| human|
 been work on my n...| 6 member band cou...| human|
 bello recuerdo en...| no soy un angel p...| human|
 benpersh svdate t...| person trainer tr...| human|
 best place to buy...| | bot|
 bfl grind davibel...| i am a brazilian ...| human|
 bighugh53 congrat...| i ve wrestl for t...| human|
 bindasbhidu dishp...| film actor artist...| human|
 birdizlif adrienn...| specialti mail or...| human|
 brand lesson ryan...| digit strategist ...| human|
 cafe arcoiri t co...| | bot|
 captulo iii ser u...| siempr estamo al ...| human|
 cari nation caitt...| carrieread specia...| human|
 chantalsimonato s...| friend of the wor...| human|
 check out my gig ...| real estat invest...| bot|
 check out thi cam...| i am of the wind ...| human|
 check out thi gre...| execut director t...| human|
 check the trend n...| keep a smile thi ...| human|
 china pose a grea...| interest in cyber...| human|
 chok t co pwmisln...| | human|
 christian pulis o...| stay up to the mi...| human|
 clementychow corn...| neurogenet counse...| human|
 cole fan realli s...| | human|
 cori bush is the ...| look for the perf...| human|
 cozdkram sit at m...| dad digit content...| human|
 d maydabeast good...| | bot|
 daft punk epilogu...| e d d a cratif | bot|
 damn the nicki st...| my privat diari t...| human|
 danieljgilli jane...| tooth and nail | human|
 dear develop and ...| engelsystem infor...| human|
 defeatbytweet jen...| | human|
 destroy what dest...| come in | bot|
 dickwhitmanbtc fa...| fri chicken deliv...| human|
 doc hillz cassett...| it a long stori | human|
 downey soccer dow...| | human|
 download tennispa...| | human|
 dpjhodg becom mor...| bear queen and le...| human|
 enter the goatbla...| | bot|
 entretenimiento v...| periodista y cond...| human|
 ever get so damn ...| just aother linub...| human|
 evydelaguila the ...| blacklivesmatt st...| human|
 facebook hat rea ...| philosophi vs con...| hu

In [0]:
# This function defines the general pipeline for logistic regression
def logistic_regression_pipeline(train, 
                                 target_variable, 
                                 with_std=True,
                                 with_mean=True,
                                 k_fold=5):

    # Configure a logistic regression pipeline, which consists of the following stages: 
    stage_1 = RegexTokenizer(inputCol="tweet_text", outputCol="tokens", pattern="\\W")
    # define stage 2: remove the stop words
    stage_2 = StopWordsRemover(inputCol="tokens", outputCol="filtered_words")
    # define stage 3: create a word vector of the size 100
    stage_3 = Word2Vec(inputCol="filtered_words", outputCol="feature_vector", vectorSize=100)

    # define stage 4: tokenize the description
    stage_4 = RegexTokenizer(inputCol="description", outputCol="tokens_des", pattern="\\W")
    # define stage 2: remove the stop words
    stage_5 = StopWordsRemover(inputCol="tokens_des", outputCol="filtered_words_des")
    # define stage 3: create a word vector of the size 100
    stage_6 = Word2Vec(inputCol="filtered_words_des", outputCol="feature_vector_des", vectorSize=100)
   

    # 1.c Indexing the target column (i.e., transform human/bot into 0/1) and rename it as "label"
    label_indexer = StringIndexer(inputCol = target_variable, outputCol = "label")
    
    # 1.d Assemble all the features (both one-hot-encoded categorical and numerical) into a single vector
    assembler = VectorAssembler(inputCols=["feature_vector","feature_vector_des"] , outputCol="features")

    # 2.a Create the StandardScaler
    # scaler = StandardScaler(inputCol=assembler.getOutputCol(), outputCol="std_"+assembler.getOutputCol(), withStd=with_std, withMean=with_mean)
    # ...

    # 3 Populate the stages of the pipeline with all the preprocessing steps
    stages = [stage_1] + [stage_2] + [stage_3] + [stage_4] + [stage_5] + [stage_6] + [label_indexer] + [assembler]  # + [scaler] + ...

    # 4. Create the logistic regression transformer
    log_reg = LogisticRegression(featuresCol="features", labelCol="label", maxIter=100) # change `featuresCol=std_features` if scaler is used
    # 5. Add the logistic regression transformer to the pipeline stages (i.e., the last one)
    stages += [log_reg]

    # 6. Set up the pipeline
    pipeline = Pipeline(stages=stages)

    # We use a ParamGridBuilder to construct a grid of parameters to search over.
    # A CrossValidator requires an Estimator, a set of Estimator ParamMaps, and an Evaluator.
    # We use a ParamGridBuilder to construct a grid of parameters to search over.
    # With 3 values for log_reg.regParam ($\lambda$) and 3 values for log_reg.elasticNetParam ($\alpha$),
    # this grid will have 3 x 3 = 9 parameter settings for CrossValidator to choose from.
    param_grid = ParamGridBuilder()\
    .addGrid(log_reg.regParam, [0.0, 0.05, 0.1]) \
    .addGrid(log_reg.elasticNetParam, [0.0, 0.5, 1.0]) \
    .build()
    
    cross_val = CrossValidator(estimator=pipeline, 
                               estimatorParamMaps=param_grid,
                               evaluator=BinaryClassificationEvaluator(metricName="areaUnderROC"), # default = "areaUnderROC", alternatively "areaUnderPR"
                               numFolds=k_fold,
                               collectSubModels=True # this flag allows us to store ALL the models trained during k-fold cross validation
                               )

    # Run cross-validation, and choose the best set of parameters.
    cv_model = cross_val.fit(train)

    return cv_model

In [0]:
cv_model = logistic_regression_pipeline(tweet_train_df, TARGET_VARIABLE)

MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.

In [0]:
# This function summarizes all the models trained during k-fold cross validation
def summarize_all_models(cv_models):
    for k, models in enumerate(cv_models):
        print("*************** Fold #{:d} ***************\n".format(k+1))
        for i, m in enumerate(models):
            print("--- Model #{:d} out of {:d} ---".format(i+1, len(models)))
            print("\tParameters: lambda=[{:.3f}]; alpha=[{:.3f}] ".format(m.stages[-1]._java_obj.getRegParam(), m.stages[-1]._java_obj.getElasticNetParam()))
            print("\tModel summary: {}\n".format(m.stages[-1]))
        print("***************************************\n")

In [0]:
# Call the function above|
summarize_all_models(cv_model.subModels)

In [0]:
for i, avg_roc_auc in enumerate(cv_model.avgMetrics):
    print("Avg. ROC AUC computed across k-fold cross validation for model setting #{:d}: {:.3f}".format(i+1, avg_roc_auc))


In [0]:

print("Best model according to k-fold cross validation: lambda=[{:.3f}]; alfa=[{:.3f}]".
      format(cv_model.bestModel.stages[-1]._java_obj.getRegParam(), 
             cv_model.bestModel.stages[-1]._java_obj.getElasticNetParam(),
             )
      )
print(cv_model.bestModel.stages[-1])
# `bestModel` is the best resulting model according to k-fold cross validation, which is also entirely retrained on the whole `train_df`
training_result = cv_model.bestModel.stages[-1].summary

In [0]:
# Make predictions on the test set (`cv_model` contains the best model according to the result of k-fold cross validation)
# `test_df` will follow exactly the same pipeline defined above, and already fit to `train_df`
test_predictions = cv_model.transform(tweet_test_df)
test_predictions.select("features", "prediction", "label").show(5)

In [0]:
def evaluate_model(predictions, metric="areaUnderROC"):
    evaluator = BinaryClassificationEvaluator(metricName=metric)
    return evaluator.evaluate(predictions)

In [0]:
print("***** Training Set *****")
print("Area Under ROC Curve (ROC AUC): {:.3f}".format(training_result.areaUnderROC))
  
print("***** Test Set *****")
print("Area Under ROC Curve (ROC AUC): {:.3f}".format(evaluate_model(test_predictions)))
print("Area Under Precision-Recall Curve: {:.3f}".format(evaluate_model(test_predictions, metric="areaUnderPR")))

## Decision Tree

In [0]:
# This function defines the general pipeline for logistic regression
def decision_tree_pipeline(train, 
                           target_variable, 
                           with_std=True,
                           with_mean=True,
                           k_fold=5):


    stage_1 = RegexTokenizer(inputCol="tweet_text", outputCol="tokens", pattern="\\W")
    # define stage 2: remove the stop words
    stage_2 = StopWordsRemover(inputCol="tokens", outputCol="filtered_words")
    # define stage 3: create a word vector of the size 100
    stage_3 = Word2Vec(inputCol="filtered_words", outputCol="feature_vector", vectorSize=100)

    # define stage 4: tokenize the description
    stage_4 = RegexTokenizer(inputCol="description", outputCol="tokens_des", pattern="\\W")
    # define stage 2: remove the stop words
    stage_5 = StopWordsRemover(inputCol="tokens_des", outputCol="filtered_words_des")
    # define stage 3: create a word vector of the size 100
    stage_6 = Word2Vec(inputCol="filtered_words_des", outputCol="feature_vector_des", vectorSize=100)


    # 1.c Indexing the target column (i.e., transform it into 0/1) and rename it as "label"
    # Note that by default StringIndexer will assign the value `0` to the most frequent label, which in the case of `deposit` is `no`
    # As such, this nicely resembles the idea of having `deposit = 0` if no deposit is subscribed, or `deposit = 1` otherwise.
    label_indexer = StringIndexer(inputCol = target_variable, outputCol = "label")
    
    # 1.d Assemble all the features (both one-hot-encoded categorical and numerical) into a single vector
    assembler = VectorAssembler(inputCols=["feature_vector"]+["feature_vector_des"] , outputCol="features")

    # 2.a Create the StandardScaler
    # scaler = StandardScaler(inputCol=assembler.getOutputCol(), outputCol="std_"+assembler.getOutputCol(), withStd=with_std, withMean=with_mean)
    # ...

    # 3 Populate the stages of the pipeline with all the preprocessing steps
    stages = [stage_1] + [stage_2] + [stage_3] + [stage_4] + [stage_5] + [stage_6] + [label_indexer] + [assembler]  # + [scaler] + ...

    # Create the decision tree transformer
    dt = DecisionTreeClassifier(featuresCol="features", labelCol="label") # change `featuresCol=std_features` if scaler is used

    # 5. Add the decision tree transformer to the pipeline stages (i.e., the last one)
    stages += [dt]

    # 6. Set up the pipeline
    pipeline = Pipeline(stages=stages)

    # We use a ParamGridBuilder to construct a grid of parameters to search over.
    # A CrossValidator requires an Estimator, a set of Estimator ParamMaps, and an Evaluator.
    # We use a ParamGridBuilder to construct a grid of parameters to search over.
    # With 3 values for dt.maxDepth and 2 values for dt.impurity
    # this grid will have 3 x 2 = 9 parameter settings for CrossValidator to choose from.
    param_grid = ParamGridBuilder()\
    .addGrid(dt.maxDepth, [3, 5, 8]) \
    .addGrid(dt.impurity, ["gini", "entropy"]) \
    .build()
    
    cross_val = CrossValidator(estimator=pipeline, 
                               estimatorParamMaps=param_grid,
                               evaluator=BinaryClassificationEvaluator(metricName="areaUnderROC"), # default = "areaUnderROC", alternatively "areaUnderPR"
                               numFolds=k_fold,
                               collectSubModels=True # this flag allows us to store ALL the models trained during k-fold cross validation
                               )

    # Run cross-validation, and choose the best set of parameters.
    cv_model = cross_val.fit(train)

    return cv_model

In [0]:
cv_model = decision_tree_pipeline(tweet_train_df, TARGET_VARIABLE)

MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.

In [0]:
# This function summarizes all the models trained during k-fold cross validation
def summarize_all_models(cv_models):
    for k, models in enumerate(cv_models):
        print("*************** Fold #{:d} ***************\n".format(k+1))
        for i, m in enumerate(models):
            print("--- Model #{:d} out of {:d} ---".format(i+1, len(models)))
            print("\tParameters: maxDept=[{:d}]; impurity=[{:s}] ".format(m.stages[-1]._java_obj.getMaxDepth(), m.stages[-1]._java_obj.getImpurity()))
            print("\tModel summary: {}\n".format(m.stages[-1]))
        print("***********************

In [0]:
summarize_all_models(cv_model.subModels)

In [0]:
for i, avg_roc_auc in enumerate(cv_model.avgMetrics):
    print("Avg. ROC AUC computed across k-fold cross validation for model setting #{:d}: {:.3f}".format(i+1, avg_roc_auc))

print("Best model according to k-fold cross validation: maxDept=[{:d}]; impurity=[{:s}]".
      format(cv_model.bestModel.stages[-1]._java_obj.getMaxDepth(), 
             cv_model.bestModel.stages[-1]._java_obj.getImpurity(),
             )
      )
print(cv_model.bestModel.stages[-1])
training_result = cv_model.bestModel.stages[-1].summary

In [0]:
# Make predictions on the test set (`cv_model` contains the best model according to the result of k-fold cross validation)
# `test_df` will follow exactly the same pipeline defined above, and already fit to `train_df`
test_predictions = cv_model.transform(tweet_test_df)
test_predictions.select("features", "prediction", "label").show(5)

In [0]:
print("***** Training Set *****")
print("Area Under ROC Curve (ROC AUC): {:.3f}".format(training_result.areaUnderROC))
  
print("***** Test Set *****")
print("Area Under ROC Curve (ROC AUC): {:.3f}".format(evaluate_model(test_predictions)))
print("Area Under Precision-Recall Curve: {:.3f}".format(evaluate_model(test_predictions, metric="areaUnderPR")))

## Random Forests

In [0]:
# This function defines the general pipeline for logistic regression
def random_forest_pipeline(train, 
                           target_variable, 
                           with_std=True,
                           with_mean=True,
                           k_fold=5):

    # define stage 1: tokenize the tweets
    stage_1 = RegexTokenizer(inputCol="tweet_text", outputCol="tokens", pattern="\\W")
    # define stage 2: remove the stop words
    stage_2 = StopWordsRemover(inputCol="tokens", outputCol="filtered_words")
    # define stage 3: create a word vector of the size 100
    stage_3 = Word2Vec(inputCol="filtered_words", outputCol="feature_vector", vectorSize=100)

    # define stage 4: tokenize the description
    stage_4 = RegexTokenizer(inputCol="description", outputCol="tokens_des", pattern="\\W")
    # define stage 2: remove the stop words
    stage_5 = StopWordsRemover(inputCol="tokens_des", outputCol="filtered_words_des")
    # define stage 3: create a word vector of the size 100
    stage_6 = Word2Vec(inputCol="filtered_words_des", outputCol="feature_vector_des", vectorSize=100)
   

    # Indexing the target column (i.e., transform it into 0/1) and rename it as "label"
    # Note that by default StringIndexer will assign the value `0` to the most frequent label, which in the case of `deposit` is `no`
    # As such, this nicely resembles the idea of having `deposit = 0` if no deposit is subscribed, or `deposit = 1` otherwise.
    label_indexer = StringIndexer(inputCol = target_variable, outputCol = "label")
    
    # 1.d Assemble all the features (both one-hot-encoded categorical and numerical) into a single vector
    assembler = VectorAssembler(inputCols=["feature_vector"]+["feature_vector_des"] , outputCol="features")

    # 2.a Create the StandardScaler
    # scaler = StandardScaler(inputCol=assembler.getOutputCol(), outputCol="std_"+assembler.getOutputCol(), withStd=with_std, withMean=with_mean)
    # ...

    # 3 Populate the stages of the pipeline with all the preprocessing steps
    stages = [stage_1] + [stage_2] + [stage_3] + [stage_4] + [stage_5] + [stage_6] + [label_indexer] + [assembler]  # + [scaler] + ...

    # Create the random forest transformer
    rf = RandomForestClassifier(featuresCol="features", labelCol="label") # change `featuresCol=std_features` if scaler is used

    # 5. Add the random forest transformer to the pipeline stages (i.e., the last one)
    stages += [rf]

    # 6. Set up the pipeline
    pipeline = Pipeline(stages=stages)

    # We use a ParamGridBuilder to construct a grid of parameters to search over.
    # A CrossValidator requires an Estimator, a set of Estimator ParamMaps, and an Evaluator.
    # We use a ParamGridBuilder to construct a grid of parameters to search over.
    # With 3 values for rf.maxDepth and 3 values for rf.numTrees
    # this grid will have 3 x 3 = 9 parameter settings for CrossValidator to choose from.
    param_grid = ParamGridBuilder()\
    .addGrid(rf.maxDepth, [3, 5, 8]) \
    .addGrid(rf.numTrees, [10, 50, 100]) \
    .build()
    cross_val = CrossValidator(estimator=pipeline, 
                               estimatorParamMaps=param_grid,
                               evaluator=BinaryClassificationEvaluator(metricName="areaUnderROC"), # default = "areaUnderROC", alternatively "areaUnderPR"
                               numFolds=k_fold,
                               collectSubModels=True # this flag allows us to store ALL the models trained during k-fold cross validation
                               )

    # Run cross-validation, and choose the best set of parameters.
    cv_model = cross_val.fit(train)

    return cv_model

In [0]:
cv_model = random_forest_pipeline(tweet_train_df, TARGET_VARIABLE)
for i, avg_roc_auc in enumerate(cv_model.avgMetrics):
    print("Avg. ROC AUC computed across k-fold cross validation for model setting #{:d}: {:.3f}".format(i+1, avg_roc_auc))
print("Best model according to k-fold cross validation: maxDept=[{:d}]".
      format(cv_model.bestModel.stages[-1]._java_obj.getMaxDepth(), 
             )
      )
print(cv_model.bestModel.stages[-1])
training_result = cv_model.bestModel.stages[-1].summary

MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.

In [0]:
# Make predictions on the test set (`cv_model` contains the best model according to the result of k-fold cross validation)
# `test_df` will follow exactly the same pipeline defined above, and already fit to `train_df`
test_predictions = cv_model.transform(tweet_test_df)
test_predictions.select("features", "prediction", "label").show(5)

In [0]:
print("***** Training Set *****")
print("Area Under ROC Curve (ROC AUC): {:.3f}".format(training_result.areaUnderROC))
  
print("***** Test Set *****")
print("Area Under ROC Curve (ROC AUC): {:.3f}".format(evaluate_model(test_predictions)))
print("Area Under Precision-Recall Curve: {:.3f}".format(evaluate_model(test_predictions, metric="areaUnderPR")))